# Custom Suite: new capabilities

English data

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

from nltk.corpus import wordnet as wn

import checklist
import spacy
import itertools
import pandas as pd

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

In [2]:
#conda install -c conda-forge jupyter_contrib_nbextensions

## Exploring and preparing the datasets

In [3]:
ironic=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/French-Italian-and-English-Corpora/English_Corpus.csv', header=None, error_bad_lines=False)
ironic

b'Skipping line 2: expected 1 fields, saw 3\nSkipping line 3: expected 1 fields, saw 2\nSkipping line 6: expected 1 fields, saw 2\nSkipping line 7: expected 1 fields, saw 3\nSkipping line 8: expected 1 fields, saw 2\nSkipping line 9: expected 1 fields, saw 5\nSkipping line 13: expected 1 fields, saw 2\nSkipping line 25: expected 1 fields, saw 2\nSkipping line 30: expected 1 fields, saw 2\nSkipping line 34: expected 1 fields, saw 2\nSkipping line 35: expected 1 fields, saw 2\nSkipping line 38: expected 1 fields, saw 3\nSkipping line 40: expected 1 fields, saw 2\nSkipping line 46: expected 1 fields, saw 2\nSkipping line 48: expected 1 fields, saw 2\nSkipping line 49: expected 1 fields, saw 2\nSkipping line 63: expected 1 fields, saw 4\nSkipping line 67: expected 1 fields, saw 2\nSkipping line 70: expected 1 fields, saw 2\nSkipping line 76: expected 1 fields, saw 3\nSkipping line 80: expected 1 fields, saw 6\nSkipping line 81: expected 1 fields, saw 3\nSkipping line 87: expected 1 fields,

,0
0,2 NEW SONGS NEXT WEEK? I'M NOT READY @justinbi...
1,*is ironically in love w Justin Bieber until i...
2,House passes Boehner/Obama budget. Raises spen...
3,How ironic is it that Obama sends BILLIONS of ...
4,@JBaileysMom I find it ironic since Muslim Oba...
...,...
326,We shouldn't sleep. We should br dedicated twe...
327,Morsi fell before Bashar. #Syria @fsa_media_hub
328,-Is It with N or L? Let me check... -It's with...
329,"Saudi position had not changed. ""There has to ..."


In [4]:
emoji_ironyHashtags=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/SemEval2018-Task3/datasets/train/SemEval2018-T3-train-taskA_emoji_ironyHashtags.txt', sep='\t', header=0, error_bad_lines=False)
emoji_ironyHashtags = emoji_ironyHashtags[emoji_ironyHashtags['Label'] == 1]
emoji_ironyHashtags

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
4,5,1,I can't breathe! was chosen as the most notabl...
6,7,1,Nothing makes me happier then getting on the h...
...,...,...,...
3804,3822,1,Someone I work w doesn't let his kids believe ...
3806,3824,1,"Obama whisked away to hospital, diagnosed with..."
3809,3827,1,Thanks for shutting the city down.. #not
3810,3828,1,@flippysgardenia IKR?! don't you see? he's gon...


In [5]:
emoji_irony=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/SemEval2018-Task3/datasets/train/SemEval2018-T3-train-taskA_emoji.txt', sep='\t', header=0, error_bad_lines=False)
emoji_irony = emoji_irony[emoji_irony['Label'] == 1]
emoji_irony

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
4,5,1,I can't breathe! was chosen as the most notabl...
6,7,1,Nothing makes me happier then getting on the h...
...,...,...,...
3804,3822,1,Someone I work w doesn't let his kids believe ...
3806,3824,1,"Obama whisked away to hospital, diagnosed with..."
3809,3827,1,Thanks for shutting the city down..
3810,3828,1,@flippysgardenia IKR?! don't you see? he's gon...


In [6]:
ironic_text=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/SemEval2018-Task3/datasets/train/SemEval2018-T3-train-taskA.txt', sep='\t', header=0, error_bad_lines=False)
ironic_text = ironic_text[ironic_text['Label'] == 1]
ironic_text

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
4,5,1,I can't breathe! was chosen as the most notabl...
6,7,1,Nothing makes me happier then getting on the h...
...,...,...,...
3804,3822,1,Someone I work w doesn't let his kids believe ...
3806,3824,1,"Obama whisked away to hospital, diagnosed with..."
3809,3827,1,Thanks for shutting the city down..
3810,3828,1,@flippysgardenia IKR?! don't you see? he's gon...


In [7]:
emojis=pd.read_csv('/Users/Marta/CheckList - FBK/Evaluation_Datasets/EmoTag/data/EmoTag1200-scores.csv', header=0, error_bad_lines=False)
emojis

,unicode,emoji,name,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,1F308,🌈,rainbow,0.00,0.28,0.00,0.00,0.69,0.06,0.22,0.33
1,1F319,🌙,crescent moon,0.00,0.31,0.00,0.00,0.25,0.00,0.06,0.25
2,1F31A,🌚,new moon face,0.06,0.08,0.17,0.06,0.42,0.19,0.06,0.11
3,1F31E,🌞,sun with face,0.00,0.22,0.00,0.00,0.78,0.00,0.11,0.22
4,1F31F,🌟,glowing star,0.00,0.28,0.00,0.00,0.53,0.00,0.25,0.31
...,...,...,...,...,...,...,...,...,...,...,...
145,2757,❗,exclamation mark,0.44,0.42,0.31,0.42,0.08,0.17,0.81,0.11
146,2764,❤,red heart,0.00,0.36,0.00,0.00,0.69,0.00,0.14,0.67
147,27A1,➡,right arrow,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.22
148,2B05,⬅,left arrow,0.17,0.14,0.17,0.14,0.00,0.14,0.03,0.06


In [8]:
hope=pd.read_excel('/Users/Marta/CheckList - FBK/Evaluation_Datasets/Hope Speech Detection for Equality, Diversity, and Inclusion/english_hope_dev.xlsx', header=0, error_bad_lines=False)
hope

,Text,Text_Label,Label
0,WOW!!!!!!!That was so so inspiring and incredi...,Hope_speech,2
1,It's pretty simple. A large portion of the wor...,Hope_speech,2
2,This world needs love,Hope_speech,2
3,I'm a Buddhist...! ALL LIVES MATTER...!,Hope_speech,2
4,Im only 11 but I LOVE science and math I want ...,Hope_speech,2
...,...,...,...
267,i’m actually about to start my college on civi...,Hope_speech,2
268,@Sasha Dumse that is true. But we should ALL l...,Hope_speech,2
269,Women need to keep fighting,Hope_speech,2
270,“God gave me a choice and my choice is love” t...,Hope_speech,2


## Preprocessing the data

The confidence is = 1 because these are gold standards

In [9]:
confs = [] 
for i in range(len(ironic)):  
    confs.append(1)
tdata = ironic[0]

In [10]:
labels_hope = []
confs_hope = [] 
for i in range(len(hope)):  
    confs_hope.append(1)
    labels_hope.append(2) 
tdata_hope = hope['Text']

In [11]:
nlp = spacy.load('en_core_web_sm')

/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [12]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [13]:
sentences_hope = tdata_hope
parsed_data_hope = list(nlp.pipe(sentences_hope))

In [14]:
sentences_eih = emoji_ironyHashtags['Tweet text'][:500]
sentences_ei = emoji_irony['Tweet text'][:500]
sentences_it = ironic_text['Tweet text'][:500]
parsed_data_eih = list(nlp.pipe(emoji_ironyHashtags['Tweet text'][:500]))
parsed_data_ei = list(nlp.pipe(emoji_irony['Tweet text'][:500]))
parsed_data_it = list(nlp.pipe(ironic_text['Tweet text'][:500]))

To create the templates and use all the functionalities, we create an Editor object

In [15]:
editor = checklist.editor.Editor()
editor.tg

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating a new object, TestSuite, where we'll add all the custom tests, inspired by the original notebooks plus new ones

In [16]:
suite = TestSuite()

# CheckList

### Expanding the lexicons

Exploring the lexicons.keys

In [17]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from']


In [18]:
# kind of neutral, taken by identity terms' file within AMI2020's data
women_noun = ['woman', 'sister', 'daughter', 'girl', 'wife', 'young lady', 'lady',
              'grandmother', 'mum', 'girlfriend', 'partner']#, 'bride', 'housewife', Mrs', 'Miss']

# offensive 
offensive_women_noun = ['slut', 'bitch', 'whore', 'cunt', 'spinster', 'ditsy', 'hussy', 'frigid', 
                        'witch', 'wench', 'harpy', 'trollop', 'dyke', 'skank', 'butch']

In [19]:
work_role = ['astronaut', 'academic', 'accountant', 'activist', 'administrator', 'advisor', 'agent', 'analyst','architect', 
             'assistant', 'attorney', 'artist' 'boxer', 'builder', 'businessman', 'cameraman', 'carpenter', 'clown', 
             'consultant', 'dentist', 'detective', 'developer', 'doctor', 'doorman', 'driver', 'electrician', 
             'economist', 'editor', 'educator', 'entrepreneur', 'executive',
             'engineer', 'farmer',  'fighter', 'fireman', 'footballer', 'foreman', 
             'historian', 'hitter', 'intern', 'interpreter', 'investigator', 'investor', 'gardener', 'housekeeper', 
             'journalist', 'librarian', 'lifeguard', 'magician', 'mechanic', 'nun', 'nurse', 'painter', 'paramedic', 
             'photographer', 'pilot', 'police', 'poet', 'postman', 'priest', 'professor', 'ranger', 'repairman', 'reporter', 
             'salesman', 'scientist', 'secretary', 'singer', 'soldier', 'student', 'surgeon', 'teacher', 'waiter', 'writer', 
             'attendant', 'officer', 'player', 'organizer', 'quarterback', 'shooter']

# These sterotyped lists are taken from the file male_occupations.txt and female_occupations.txt from the WiNo dataset 
fem_work_role = ['attendant', 'cashier', 'teacher', 'nurse', 'assistant', 'secretary', 'auditor', 'cleaner', 
                 'receptionist', 'clerk', 'counselor', 'designer', 'hairdresser', 'writer', 'housekeeper', 
                 'baker', 'accountant', 'editor', 'librarian', 'tailor']

male_work_role = ['driver', 'supervisor', 'janitor', 'cook', 'mover', 'laborer', 'construction worker',
                  'chief', 'developer', 'carpenter', 'manager', 'lawyer', 'farmer', 'salesperson', 'physician',
                  'guard', 'analyst', 'mechanic', 'sheriff', 'CEO']

In [20]:
editor.add_lexicon('women_noun', women_noun, overwrite=True)
editor.add_lexicon('offensive_women_noun', offensive_women_noun, overwrite=True)
editor.add_lexicon('work_role', work_role, overwrite=True)
editor.add_lexicon('fem_work_role', fem_work_role, overwrite=True)
editor.add_lexicon('male_work_role', male_work_role, overwrite=True)

In [21]:
pos_adj = [                                                                             
    'adorable', 'amazing', 'astonishing', 'attractive', 'awesome', 'beautiful',
    'brilliant', 'caring', 'committed', 'cool', 'enthusiastic', 'excellent', 'exceptional',
    'extraordinary', 'fantastic', 'fun', 'good', 'great', 'happy', 'honest', 'impressive', 'incredible',
    'inspiring', 'intelligent', 'lovely', 'nice', #'smart', 
    'successful', 'sweet', 'trustworthy', 'wise', 'wonderful']

neg_adj = [
    'absolute', 'aggressive', 'angry', 'annoying', 'arrogant', 
    'awful', 'bad', 'bitchy', 'bloody', 'boring', 'bossy', 'creepy', 'cruel', 
    'damn', 'dead', 'depressing', 'difficult', 'dirty', 'disappointing', 'disgusting', 
    'dreadful', 'emotional', 'evil', 'fake', 'fat', 'feisty', 'frustrating', 'fucking', 
    'giant', 'hormonal', 'horrible', 'hysterical', 'illogical', 'incompetent', 
    'irrational', 'irritating', 'lame', 'lousy', 'lying', 'nasty', 'poor', 
    'rough', 'sad', 'sassy', 'shameless', 'shitty', 'stupid', 'terrible', 'terrific', 
    'twatty', 'ugly', 'unhappy', 'wanky', 'weak', 'weird', 'whining', 'wicked', 'worst']

neutral_adj = [                                           
    'american', 'armenian','atheist', 'australian', 'black', 'british', 'christian', 'commercial', 
    'english', 'gay', 'immigrant', 'independent', 'independent', 'indian', 'indian', 'intellectual','busy', 
    'international', 'israeli', 'italian', 'jewish', 'married', 'muslim', 'old', 'poor',
    'private', 'rich', 'russian', 'single', 'tall', 'unmarried', 'white', 'working', 'young']

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [22]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love', 'admire',
                   'respect', 'adore', 'support', 'care for', 'fancy', 'treasure', 'trust']

neg_verb_present = ['hate', 'dislike', 'regret', 'dread', 'despise', 'blame', 'hurt', 'envy', 'pity']

neutral_verb_present = ['see', 'find', 'miss', 'understand', 'believe', 'remember', 'talk to']

pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 
                 'respected', 'adored', 'supported', 'cared for', 'treasured', 'trusted']

neg_verb_past = ['hated', 'disliked', 'regretted', 'dreaded', 'despised','blamed', 'hurt', 'envied', 'pitied']

neutral_verb_past = ['saw', 'found', 'missed', 'understood', 'believed', 'remembered', 'talked to']

editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

In [23]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'especially',
              'exceptionally', 'utterly', 'rather', 'totally', 'particularly',
              'remarkably', 'pretty', 'wonderfully', 'completely',
              'entirely', 'undeniably', 'highly']

editor.add_lexicon('intens_adj', intens_adj, overwrite=True)

In [24]:
intens_verb = ['really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 
               'highly', 'definitely', 'certainly', 'honestly', 'strongly', 'sincerely']
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'a little', 'a bit', 'slightly']

editor.add_lexicon('intens_verb', intens_verb, overwrite=True)
editor.add_lexicon('reducer_adj', reducer_adj, overwrite=True)

Original and new lexicons keys

In [25]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from', 'women_noun', 'offensive_women_noun', 'work_role', 'fem_work_role', 'male_work_role', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb', 'intens_adj', 'intens_verb', 'reducer_adj']


## Capability 8: Taxonomy related to the Task of Sentiment Analysis

In [26]:
synonyms_pos = [('amazing', 'incredible'), ('fabulous', 'fantastic'), ('wonderful', 'extraordinary'), 
                ('beautiful', 'handsome'), ('gorgeous', 'magnificent'), ('stunning', 'dazzling'),
                ('brave', 'fearless'), ('happy', 'joyful')]
synonyms_neg = [('dull', 'boring'), ('dumb', 'stupid'), ('dreary', 'tedious'),
                ('moody', 'temperamental'), ('irritable', 'touchy'), 
                ('ugly', 'nasty'), ('horrible', 'terrifying'), ('gross', 'repugnant')]

With these, we can create simple tests, where we expect the model to recognize these synonyms.  


In [27]:
import re
def change_pos_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_pos:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [28]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:25], be=['is'], syn=synonyms_pos, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is amazing.',
 'Mary is fabulous.',
 'Mary is wonderful.',
 'Mary is beautiful.',
 'Mary is gorgeous.',
 'Mary is stunning.',
 'Mary is brave.',
 'Mary is happy.',
 'Elizabeth is amazing.',
 'Elizabeth is fabulous.',
 'Elizabeth is wonderful.',
 'Elizabeth is beautiful.',
 'Elizabeth is gorgeous.',
 'Elizabeth is stunning.',
 'Elizabeth is brave.',
 'Elizabeth is happy.',
 'Margaret is amazing.',
 'Margaret is fabulous.',
 'Margaret is wonderful.',
 'Margaret is beautiful.',
 'Margaret is gorgeous.',
 'Margaret is stunning.',
 'Margaret is brave.',
 'Margaret is happy.',
 'Sarah is amazing.',
 'Sarah is fabulous.',
 'Sarah is wonderful.',
 'Sarah is beautiful.',
 'Sarah is gorgeous.',
 'Sarah is stunning.',
 'Sarah is brave.',
 'Sarah is happy.',
 'Susan is amazing.',
 'Susan is fabulous.',
 'Susan is wonderful.',
 'Susan is beautiful.',
 'Susan is gorgeous.',
 'Susan is stunning.',
 'Susan is brave.',
 'Susan is happy.',
 'Barbara is amazing.',
 'Barbara is fabulous.',
 'Barbar

In [29]:
ret1 = Perturb.perturb(t1.data, change_pos_adj, keep_original=True)
ret1.data

[['Mary is amazing.', 'Mary is incredible.'],
 ['Mary is fabulous.', 'Mary is fantastic.'],
 ['Mary is wonderful.', 'Mary is extraordinary.'],
 ['Mary is beautiful.', 'Mary is handsome.'],
 ['Mary is gorgeous.', 'Mary is magnificent.'],
 ['Mary is stunning.', 'Mary is dazzling.'],
 ['Mary is brave.', 'Mary is fearless.'],
 ['Mary is happy.', 'Mary is joyful.'],
 ['Elizabeth is amazing.', 'Elizabeth is incredible.'],
 ['Elizabeth is fabulous.', 'Elizabeth is fantastic.'],
 ['Elizabeth is wonderful.', 'Elizabeth is extraordinary.'],
 ['Elizabeth is beautiful.', 'Elizabeth is handsome.'],
 ['Elizabeth is gorgeous.', 'Elizabeth is magnificent.'],
 ['Elizabeth is stunning.', 'Elizabeth is dazzling.'],
 ['Elizabeth is brave.', 'Elizabeth is fearless.'],
 ['Elizabeth is happy.', 'Elizabeth is joyful.'],
 ['Margaret is amazing.', 'Margaret is incredible.'],
 ['Margaret is fabulous.', 'Margaret is fantastic.'],
 ['Margaret is wonderful.', 'Margaret is extraordinary.'],
 ['Margaret is beautiful.

In [30]:
test = INV(**ret1)

name = 'she is adj vs she is positive synonym' 
desc = 'Simple template where adjs are replaced with positive synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

In [31]:
import re
def change_neg_adj(x, *args, **kwargs):
    ret = []
    for p in synonyms_neg:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)]) 
        #if re.search(r'\b%s\b' % p[1], x):
        #    ret.extend([re.sub(r'\b%s\b' % p[1], p[0], x)]) 
    return ret

In [32]:
t1 = editor.template('{fem} {be} {syn[0]}.', fem=editor.template('{female}').data[:25], be=['is'], syn=synonyms_neg, labels=0, remove_duplicates=True, save=True)

t1.data

['Mary is dull.',
 'Mary is dumb.',
 'Mary is dreary.',
 'Mary is moody.',
 'Mary is irritable.',
 'Mary is ugly.',
 'Mary is horrible.',
 'Mary is gross.',
 'Elizabeth is dull.',
 'Elizabeth is dumb.',
 'Elizabeth is dreary.',
 'Elizabeth is moody.',
 'Elizabeth is irritable.',
 'Elizabeth is ugly.',
 'Elizabeth is horrible.',
 'Elizabeth is gross.',
 'Margaret is dull.',
 'Margaret is dumb.',
 'Margaret is dreary.',
 'Margaret is moody.',
 'Margaret is irritable.',
 'Margaret is ugly.',
 'Margaret is horrible.',
 'Margaret is gross.',
 'Sarah is dull.',
 'Sarah is dumb.',
 'Sarah is dreary.',
 'Sarah is moody.',
 'Sarah is irritable.',
 'Sarah is ugly.',
 'Sarah is horrible.',
 'Sarah is gross.',
 'Susan is dull.',
 'Susan is dumb.',
 'Susan is dreary.',
 'Susan is moody.',
 'Susan is irritable.',
 'Susan is ugly.',
 'Susan is horrible.',
 'Susan is gross.',
 'Barbara is dull.',
 'Barbara is dumb.',
 'Barbara is dreary.',
 'Barbara is moody.',
 'Barbara is irritable.',
 'Barbara is u

In [33]:
ret1 = Perturb.perturb(t1.data, change_neg_adj, keep_original=True)
ret1.data

[['Mary is dull.', 'Mary is boring.'],
 ['Mary is dumb.', 'Mary is stupid.'],
 ['Mary is dreary.', 'Mary is tedious.'],
 ['Mary is moody.', 'Mary is temperamental.'],
 ['Mary is irritable.', 'Mary is touchy.'],
 ['Mary is ugly.', 'Mary is nasty.'],
 ['Mary is horrible.', 'Mary is terrifying.'],
 ['Mary is gross.', 'Mary is repugnant.'],
 ['Elizabeth is dull.', 'Elizabeth is boring.'],
 ['Elizabeth is dumb.', 'Elizabeth is stupid.'],
 ['Elizabeth is dreary.', 'Elizabeth is tedious.'],
 ['Elizabeth is moody.', 'Elizabeth is temperamental.'],
 ['Elizabeth is irritable.', 'Elizabeth is touchy.'],
 ['Elizabeth is ugly.', 'Elizabeth is nasty.'],
 ['Elizabeth is horrible.', 'Elizabeth is terrifying.'],
 ['Elizabeth is gross.', 'Elizabeth is repugnant.'],
 ['Margaret is dull.', 'Margaret is boring.'],
 ['Margaret is dumb.', 'Margaret is stupid.'],
 ['Margaret is dreary.', 'Margaret is tedious.'],
 ['Margaret is moody.', 'Margaret is temperamental.'],
 ['Margaret is irritable.', 'Margaret is to

In [34]:
test = INV(**ret1)

name = 'she is adj vs she is negative synonym' 
desc = 'Simple template where adjs are replaced with negative synonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

Let's do the same with antonyms:

In [35]:
antonyms = [('so beautiful', 'very ugly'), ('clever like no other ', 'the stupidest person alive'),
            ('truly confident', 'the most incapable person I know'), ('so interesting ', 'too insignificant')]

In [36]:
import re
def change_an(x, *args, **kwargs):
    ret = []
    for p in antonyms:        
        if re.search(r'\b%s\b' % p[0], x):
            ret.extend([re.sub(r'\b%s\b' % p[0], p[1], x)])  
    return ret

In [37]:
t1 = editor.template('{fem} {be} {an[0]}.', fem=editor.template('{female}').data[:25], be=['is'], an=antonyms, labels=2, remove_duplicates=True, save=True)

t1.data

['Mary is so beautiful.',
 'Mary is clever like no other .',
 'Mary is truly confident.',
 'Mary is so interesting .',
 'Elizabeth is so beautiful.',
 'Elizabeth is clever like no other .',
 'Elizabeth is truly confident.',
 'Elizabeth is so interesting .',
 'Margaret is so beautiful.',
 'Margaret is clever like no other .',
 'Margaret is truly confident.',
 'Margaret is so interesting .',
 'Sarah is so beautiful.',
 'Sarah is clever like no other .',
 'Sarah is truly confident.',
 'Sarah is so interesting .',
 'Susan is so beautiful.',
 'Susan is clever like no other .',
 'Susan is truly confident.',
 'Susan is so interesting .',
 'Barbara is so beautiful.',
 'Barbara is clever like no other .',
 'Barbara is truly confident.',
 'Barbara is so interesting .',
 'Helen is so beautiful.',
 'Helen is clever like no other .',
 'Helen is truly confident.',
 'Helen is so interesting .',
 'Anne is so beautiful.',
 'Anne is clever like no other .',
 'Anne is truly confident.',
 'Anne is so inte

In [38]:
ret1 = Perturb.perturb(t1.data, change_an, keep_original=True)

ret1.data

[['Mary is so beautiful.', 'Mary is very ugly.'],
 ['Mary is truly confident.', 'Mary is the most incapable person I know.'],
 ['Elizabeth is so beautiful.', 'Elizabeth is very ugly.'],
 ['Elizabeth is truly confident.',
  'Elizabeth is the most incapable person I know.'],
 ['Margaret is so beautiful.', 'Margaret is very ugly.'],
 ['Margaret is truly confident.',
  'Margaret is the most incapable person I know.'],
 ['Sarah is so beautiful.', 'Sarah is very ugly.'],
 ['Sarah is truly confident.', 'Sarah is the most incapable person I know.'],
 ['Susan is so beautiful.', 'Susan is very ugly.'],
 ['Susan is truly confident.', 'Susan is the most incapable person I know.'],
 ['Barbara is so beautiful.', 'Barbara is very ugly.'],
 ['Barbara is truly confident.',
  'Barbara is the most incapable person I know.'],
 ['Helen is so beautiful.', 'Helen is very ugly.'],
 ['Helen is truly confident.', 'Helen is the most incapable person I know.'],
 ['Anne is so beautiful.', 'Anne is very ugly.'],
 [

In [39]:
def changed_pred(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    return pred != orig_pred
expect_fn = Expect.pairwise(changed_pred)

In [40]:
test = DIR(**ret1, expect=expect_fn)

name = 'she is adj vs she is antonym' 
desc = 'Simple template where positive adjs are replaced with antonyms'

suite.add(test, name, 'Taxonomy', desc, overwrite=True)

## Capability 9: Coreference related to the Task of Sentiment Analysis

Basic coref
* he / she: 

`{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.`

`If {female} and {male} were alone, do you think she would reject him?'`

* his / her: 

`{male} and {female} are friends. His mom is {a:prof}.`

`If {male} and {female} were married, would his family be happy?`

* former, latter:

`{first_name} and {first_name2} are friends. The former/latter is {a:prof1}.`

In [41]:
t = editor.template('{male} and {female} are colleagues: he thinks that she is {pos_adj} but I don\'t agree: she is {neg_adj}', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {pos_adj} but I don\'t agree: he is {neg_adj}', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: he thinks that she is {neg_adj} but I don\'t agree: she is {pos_adj}', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {neg_adj} but I don\'t agree: he is {pos_adj}', nsamples=25, save=True, labels=2)

t += editor.template('{male} and {female} are colleagues: he thinks that she is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: he thinks that she is {neg_adj} and I agree', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {neg_adj} and I agree', nsamples=25, save=True, labels=0)

t += editor.template('{male} and {female} are colleagues: his feeling about her is {pos_adj}  and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: her feeling about him is {pos_adj}  and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: his feeling about her is {neg_adj} and I agree', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: her feeling about him is {neg_adj} I agree', nsamples=25, save=True, labels=0)


t += editor.template('{male} and {female} are colleagues: the former thinks that the latter is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: the latter thinks that the former is {pos_adj} and I agree', nsamples=25, save=True, labels=2)
t += editor.template('{male} and {female} are colleagues: the former thinks that the latter is {neg_adj} and I agree', nsamples=25, save=True, labels=0)
t += editor.template('{male} and {female} are colleagues: the latter thinks that the former is {neg_adj} and I agree', nsamples=25, save=True, labels=0)

t.data

["Francis and Rachel are colleagues: he thinks that she is trustworthy but I don't agree: she is poor",
 "Patrick and Kathryn are colleagues: he thinks that she is brilliant but I don't agree: she is terrific",
 "Scott and Rose are colleagues: he thinks that she is beautiful but I don't agree: she is evil",
 "James and Maria are colleagues: he thinks that she is nice but I don't agree: she is bitchy",
 "Ray and Martha are colleagues: he thinks that she is amazing but I don't agree: she is bad",
 "Tim and Julie are colleagues: he thinks that she is trustworthy but I don't agree: she is difficult",
 "Frank and Nancy are colleagues: he thinks that she is fantastic but I don't agree: she is creepy",
 "Steven and Caroline are colleagues: he thinks that she is wonderful but I don't agree: she is illogical",
 "Alan and Sara are colleagues: he thinks that she is intelligent but I don't agree: she is frustrating",
 "Tom and Sophie are colleagues: he thinks that she is caring but I don't agree: 

In [42]:
test = MFT(**t)

suite.add(test, 'opinions', 'Coref', 'Phrases that report the opinion of x(male) on y(female) and of y on x, on which the speaker agrees or disagrees.')

In [43]:
t = editor.template('{male} and {female} are colleagues: he thinks that she is {neutral_adj} and I agree', nsamples=25, save=True)
t += editor.template('{male} and {female} are colleagues: she thinks that he is {neutral_adj} and I agree', nsamples=25, save=True)

In [44]:
t.data

['Tony and Edith are colleagues: he thinks that she is independent and I agree',
 'Nick and Joan are colleagues: he thinks that she is independent and I agree',
 'David and Barbara are colleagues: he thinks that she is british and I agree',
 'Jack and Dorothy are colleagues: he thinks that she is independent and I agree',
 'Eric and Carolyn are colleagues: he thinks that she is independent and I agree',
 'Alex and Elizabeth are colleagues: he thinks that she is indian and I agree',
 'Philip and Stephanie are colleagues: he thinks that she is busy and I agree',
 'Joe and Kathy are colleagues: he thinks that she is italian and I agree',
 'Chris and Michelle are colleagues: he thinks that she is single and I agree',
 'Bruce and Anne are colleagues: he thinks that she is unmarried and I agree',
 'Ben and Eleanor are colleagues: he thinks that she is private and I agree',
 'Alexander and Robin are colleagues: he thinks that she is independent and I agree',
 'Kenneth and Victoria are colleag

In [45]:
is_not_0 = lambda x, pred, *args: pred != 0

In [46]:
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)

suite.add(test, 'neutral opinions', 'Coref', 'Phrases that report the opinion of x(male) on y(female) and of y on x, on which the speaker agrees.')

## Capability 10: Logic

**NB** We describe example of testing of this capability, but don't implement it in the suite cause it is difficult and not needed in the task of Sentiment Analysis. 

_Ability to handle symmetry, consistency, and conjunctions_

* A or B is not the same as C and D => `Is {first_name} {last_name} {a:p1} or {a:p2}? {...}`
* A or B is not the same as A and B => `{...} Is {first_name} {last_name} simultaneously {a:p1} and {a:p2}?`
* A and / or B is the same as B and / or A => `Is {first_name} {last_name} {a:p1} {andor} {a:p2}?`
* A {nationality} {profession} = a {profession} and {nationality} => `Is {first_name} {last_name} {a:nat} {p1}?`

Reflexivity
* (q, q) should be duplicate

Simmetry
* f(a, b) = f(b, a)

Implications
* f(x, a) = 1 and f(x, b) = 1 => f(a, b) = 1
* f(x, a) = 1 and f(x, b) = 0 => f(a, b) = 0

Only used (x, a, b) such that (x, a) and (x, b) in val dataset and (a, b) is not. Expectation function filters out examples where f(x, a) or f(x, b) are incorrect

## Capability 11: Sarcasm & Irony

In [47]:
is_not_1 = lambda x, pred, *args: pred != 1

### Ironic tweet, SemEval 2018

In [48]:
dataset_it=[]
for item in sentences_it:
    dataset_it.append(item)

In [49]:
dataset_eih=[]
for item in sentences_eih:
    dataset_eih.append(item)

#### as DIR

To understand if the prediction change from the "plain" text to the text with emojis and #sarcastic hashtags

In [50]:
# We're  expecting the score to change 
monotonic_label = Expect.monotonic(tolerance=0.05)
preds = lambda pred, *args, **kwargs: pred == 0 or pred == 1 or pred == 2 
monotonic_label = Expect.slice_pairwise(monotonic_label, preds)

In [51]:
t=[]
for i in range(len(dataset_it)):
    t.append([dataset_it[i],dataset_eih[i]])

In [52]:
t

[['Sweet United Nations video. Just in time for Christmas. #imagine #NoReligion  http://t.co/fej2v3OUBR',
  'Sweet United Nations video. Just in time for Christmas. #imagine #NoReligion #irony http://t.co/fej2v3OUBR'],
 ["@mrdahl87 We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing    ;)",
  "@mrdahl87 We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing #Sarcasm   ;)"],
 ['Hey there! Nice to see you Minnesota/ND Winter Weather ',
  'Hey there! Nice to see you Minnesota/ND Winter Weather #Not'],
 ["I can't breathe! was chosen as the most notable quote of the year in an annual list released by a Yale University librarian ",
  "I can't breathe! was chosen as the most notable quote of the year in an annual list released by a Yale University librarian #irony"],
 ['Nothing makes me happier then getting on the highway and seeing break lights light up like a Christmas tree.. '

In [53]:
test = DIR(t, monotonic_label)

suite.add(test, 'role of #sarcastic hashtag and emojis', 'Sarcasm & Irony', 'Test is composed of pairs of sentences (x1, x2): the difference is in the presence of the hashtag #sarcastic and emojis', overwrite=True)

## Capability 12: Social-specific lexicons

### Emojis

#### As MFT

In [54]:
hate_emojis=[]
for i in range(len(emojis)):
    if(emojis['anger'][i]>0.5 or emojis['disgust'][i]>0.5):
        hate_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['anger'][i])
        print(emojis['disgust'][i])

👎
0.5
0.67
💩
0.14
0.64
😒
0.58
0.64
😖
0.22
0.67
😠
1.0
0.56
😡
1.0
0.56
😣
0.31
0.67
😤
0.75
0.64
😫
0.36
0.64
😷
0.03
0.53
🙅
0.47
0.53
❌
0.5
0.58


In [55]:
hate_emojis

['👎', '💩', '😒', '😖', '😠', '😡', '😣', '😤', '😫', '😷', '🙅', '❌']

In [56]:
test = MFT(hate_emojis, labels=0)
suite.add(test, 'hate/disgust emojis', 'Social', 'Simple tests involving negative emojis', overwrite=True)

In [57]:
happy_emojis=[]
for i in range(len(emojis)):
    if(emojis['joy'][i]>0.5 and emojis['trust'][i]>0.5):
        happy_emojis.append(emojis['emoji'][i])
        print(emojis['emoji'][i])
        print(emojis['joy'][i])
        print(emojis['trust'][i])

🌹
0.56
0.72
🍻
0.72
0.53
👏
0.64
0.53
💋
0.78
0.69
💐
0.69
0.58
💓
0.61
0.67
💕
0.83
0.75
💖
0.89
0.53
💘
0.67
0.64
💞
0.83
0.61
💯
0.64
0.53
😌
0.81
0.61
😘
0.72
0.61
😙
0.83
0.83
😚
0.86
0.67
♥
0.72
0.56
❤
0.69
0.67


In [58]:
happy_emojis

['🌹',
 '🍻',
 '👏',
 '💋',
 '💐',
 '💓',
 '💕',
 '💖',
 '💘',
 '💞',
 '💯',
 '😌',
 '😘',
 '😙',
 '😚',
 '♥',
 '❤']

In [59]:
test = MFT(happy_emojis, labels=2)
suite.add(test, 'happy emojis', 'Social', 'Simple tests involving only positive emojis', overwrite=True)

#### as DIR

Combining templates + emojis

In [60]:
# We're  expecting the score to raise 
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.05)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [61]:
t = editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{it} {be} {a:pos_adj} {women_noun}.', '{it} {be} {a:pos_adj} {women_noun} {hp_e}{hp_e}'] , hp_e=happy_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)
t += editor.template(['{i} {pos_verb} {the} {women_noun}.', '{i} {pos_verb} {the} {women_noun} {hp_e}{hp_e}'], hp_e=happy_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=2, save=True)

test = DIR(t.data, monotonic_label, templates=t.templates)

suite.add(test, 'emoji intensifiers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add an intensifier emoji', overwrite=True)

In [62]:
t.data

[['She was a happy daughter.', 'She was a happy daughter ♥'],
 ['She is a wonderful wife.', 'She is a wonderful wife 👏'],
 ['She is an awesome grandmother.', 'She is an awesome grandmother 💘'],
 ['She is an adorable sister.', 'She is an adorable sister 💯'],
 ['She is a fun wife.', 'She is a fun wife 🌹'],
 ['She was a beautiful woman.', 'She was a beautiful woman 🌹'],
 ['She was a fun grandmother.', 'She was a fun grandmother 💖'],
 ['She was a beautiful sister.', 'She was a beautiful sister 💞'],
 ['She is an excellent sister.', 'She is an excellent sister 😚'],
 ['She was a wonderful partner.', 'She was a wonderful partner 😌'],
 ['She is a brilliant girl.', 'She is a brilliant girl ♥'],
 ['She was a caring partner.', 'She was a caring partner ♥'],
 ['She was a wonderful lady.', 'She was a wonderful lady 💐'],
 ['She was a fantastic sister.', 'She was a fantastic sister 💕'],
 ['She was a happy wife.', 'She was a happy wife 💕'],
 ['She was an excellent young lady.', 'She was an excellent yo

In [63]:
# We're  expecting the score to fall 
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.05)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

In [64]:
t = editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {offensive_women_noun}.', '{it} {be} {a:neg_adj} {offensive_women_noun} {ht_e}{ht_e}'] , ht_e=hate_emojis, it=['She'], be=['is', 'was'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)
t += editor.template(['{i} {neg_verb} {the} {offensive_women_noun}.', '{i} {neg_verb} {the} {offensive_women_noun} {ht_e}{ht_e}'], ht_e=hate_emojis, i=['I', 'We', 'You'], the=['this', 'that'], nsamples=200, labels=0, save=True)

test = DIR(t.data, monotonic_label_down, templates=t.templates)

suite.add(test, 'emoji reducers', 'Social', 'Test is composed of pairs of sentences (x1, x2), where we add a reducer emoji', overwrite=True)

In [65]:
t.data

[['She is a whining dyke.', 'She is a whining dyke ❌'],
 ['She is a fake trollop.', 'She is a fake trollop 😖'],
 ['She was a worst dyke.', 'She was a worst dyke 😷'],
 ['She was a fake cunt.', 'She was a fake cunt 👎'],
 ['She is a hormonal cunt.', 'She is a hormonal cunt 😖'],
 ['She was a sassy whore.', 'She was a sassy whore 💩'],
 ['She is an absolute trollop.', 'She is an absolute trollop 😫'],
 ['She is a nasty bitch.', 'She is a nasty bitch 😒'],
 ['She was a poor bitch.', 'She was a poor bitch 😠'],
 ['She was an emotional skank.', 'She was an emotional skank ❌'],
 ['She is a frustrating whore.', 'She is a frustrating whore 😡'],
 ['She was an ugly trollop.', 'She was an ugly trollop 😤'],
 ['She was a wicked slut.', 'She was a wicked slut 💩'],
 ['She was a difficult whore.', 'She was a difficult whore 👎'],
 ['She was an incompetent butch.', 'She was an incompetent butch 😫'],
 ['She was a rough harpy.', 'She was a rough harpy 😒'],
 ['She is a sad whore.', 'She is a sad whore 😡'],
 ['She

## Capability 13: Hope Speech Detection

Dataset  from the competition: _Hope Speech Detection for Equality, Diversity, and Inclusion-EACL 2021_

**Although it is not a "linguistic capacity", it may be linked to the sentiment, but in a different ways as traditionally intended: finding hope and positive messages, instead of toxic ones**

#### as MFT

In [66]:
dataset=[]
for i in range(len(sentences_hope)):
    dataset.append(sentences_hope[i])
    
l_hope=[]
for i in range(len(labels_hope)):
    l_hope.append(labels_hope[i])
l_hope

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [67]:
test = MFT(dataset, labels=l_hope)
suite.add(test, 'hopeful tweets', 'Hope Speech', 'Tweets involving "support, reassurance, suggestions, inspiration"', overwrite=True)

#### as INV

In [68]:
t = Perturb.perturb(parsed_data_hope, Perturb.change_names, nsamples=200)

test = INV(t.data)

suite.add(test, 'change names', 'Hope Speech', 'Replace names with other common names', overwrite=True )

t.data

[["I can't thank you enough Debbie for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Sara for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Ashley for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Chloe for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Karen for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Amy for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Mary for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Haley for this talknI had the luck of being able to play with not just girl toys",
  "I can't thank you enough Kristen for this talknI had the luck of being able to play with not just girl toys",
 

In [69]:
t = Perturb.perturb(parsed_data_hope, Perturb.change_location, nsamples=200)

test = INV(t.data)

suite.add(test, 'change locations', 'Hope Speech', 'Replace city or country names with other cities or countries', overwrite=True)

t.data

[["India produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Morocco produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Germany produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "United Kingdom produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Japan produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Madagascar produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Japan produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "France produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Sudan produces a lot of GREAT female engineers... they like to work!!  A lot of white women don't!",
  "Venezuela produces a lot of GREAT female e

In [70]:
t = Perturb.perturb(parsed_data_hope, Perturb.change_number, nsamples=200)

test = INV(t.data)

suite.add(test, 'change numbers', 'Hope Speech', 'Replace integers with random integers within a 20% radius of the original', overwrite=True)

t.data

[['Im only 11 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery. I defintitely think that this will impower young girls like me or younger.',
  'Im only 13 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery. I defintitely think that this will impower young girls like me or younger.',
  'Im only 12 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery. I defintitely think that this will impower young girls like me or younger.',
  'Im only 12 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery. I defintitely think that this will impower young girls like me or younger.',
  'Im only 14 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery. I defintitely think that this will impower young girls like me or younger.',
  'Im only 9 but I LOVE science and math I want to be a trauma surgeon specializing in nuero surgery

## Saving tests & Exporting the suite to a file

In [71]:
suite.to_raw_file('/Users/Marta/CheckList - FBK/3_New_Capabilities.txt', n=500, seed=1)

In [72]:
for test in suite.tests:
    suite.tests[test].name = test
    suite.tests[test].description = suite.info[test]['description]']
    suite.tests[test].capability = suite.info[test]['capability']

In [73]:
path = '/Users/Marta/opt/anaconda3/lib/python3.6/site-packages/checklist/release_data/sentiment/3_New_Capabilities.pkl'
suite.save(path)